In [ ]:
# colab 사용 시 주석 풀고 mount_path 설정 후 실행
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
#colab 사용시 주석 풀고 실행
# !pip install transformers
# vscode library 설치 안되있을 경우 실행
# %pip install transformers

In [ ]:
#colab 사용시 주석 풀고 실행
# !pip install seaborn
# vscode library 설치 안되있을 경우 실행
# %pip install seaborn

In [ ]:
#colab 사용시 주석 풀고 실행
# !pip install pytorch-lightning
# vscode library 설치 안되있을 경우 실행
# %pip install pytorch-lightning

In [1]:
from _init import *

from commons import file_util, string_util

import time
import torch
import random
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.data_utils import pad_sequences

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# file_dir variable
WORK_DIR = "../../../"
IN_DIR = WORK_DIR + "resources/keyword_extract/"
OUT_MODEL_PATH = WORK_DIR + "resources/keyword_extract_model/bert_model.pt"
DELIM = "\t"
TEST_SIZE = 0.112
SETP_SIZE = 100

# Hyperparameter variable
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 32
MAX_SEQ_LEN = 256           # 내가 입력할 문장을 쪼갠 단어의 총 갯수를 의미 -> 자릿수를 맞춰주기 위해 0으로 채우는 것 padding
LEARNING_RATE = 2e-5
DROPOUT_RATE = 0.3
MODE = "min"
PATIENCE = 3
EPOCHS = 50
SEED_VAL = 0

DENCE_UNIT = 2

# 사전학습된 언어 bert model name 활성화
# BERT_MODEL_NAME = "bert-base-uncased"
# BERT_MODEL_NAME = "bert-base-multilingual-cased"
BERT_MODEL_NAME = "klue/bert-base"
# 이거는 너무 커서 안돌아감.
# BERT_MODEL_NAME = "klue/roberta-large"

c:\Users\lklkk\anaconda3\envs\sejong\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device_name = tf.test.gpu_device_name()

# GPU divice name checker
if device_name == "/device:GPU:0" :
    print("Found GPU at : {}".format(device_name))
else :
    print("GPU device not found")

Found GPU at : /device:GPU:0


In [3]:
# device setting
if torch.cuda.is_available() :
    device = torch.device("cuda")

    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU : ", torch.cuda.get_device_name(0))
else :
    device = torch.device("cpu")
    print("No GPU available, using the CPU instead.")

There are 1 GPU(s) available.
We will use the GPU :  NVIDIA GeForce RTX 3060


In [4]:
# vs code 사용 시 데이터 불러오기
# data -> list로 넘길 때 사용
def load(in_file_path: str, encoding: str, out_list: list) :
	file_paths = file_util.get_file_paths(in_file_path, True)

	for file_path in file_paths :
		in_file = file_util.open_file(file_path, encoding, "r")

		while True :
			line = in_file.readline()

			if not line :
				break

			line = file_util.preprocess(line)
			if string_util.is_empty(line, True) :
				continue

			out_list.append(line)
	in_file.close()

# 이거 활성화 시켜서 사용하면 됨.
# raw_data = []
# load(IN_DIR, ENCODING, raw_data)
# raw_data = [line.strip().split(DELIM) for line in raw_data]

# raw_df = pd.DataFrame(raw_data, columns=["train", "label"])
# print(raw_df.shape)
# raw_df.head()

In [5]:
# file_name 추출
file_paths = file_util.get_file_paths(IN_DIR, True)

for file_path in file_paths :
    file_name = file_util.get_file_name(file_path)

In [6]:
raw_df = pd.read_csv(IN_DIR + file_name, sep=DELIM, names=["train", "label"])

raw_df.shape

(12540, 2)

In [7]:
raw_df.head()

,train,label
0,### 이건 테스트 문장입니다 #,1
1,## 이건 테스트 문장입니다 ##,1
2,# 이건 테스트 문장입니다 ###,1
3,### 엠마누엘 웅가로 / 의상서,1
4,## 엠마누엘 웅가로 / 의상서 실내,1


In [8]:
# train_set, test_set 분할
# 비율 조정 해서 train 8.1, validation 0.9, test 1.0
print("여기서부터 문맥에 맞게 DATA_RATE 값 변경")
print(f"train - test : {int(raw_df.shape[0] * 0.9)}")
DATA_RATE = 11277

train_df = raw_df[:DATA_RATE]
test_df = raw_df[DATA_RATE:]

train_df.tail(), test_df.head()

여기서부터 문맥에 맞게 DATA_RATE 값 변경
train - test : 11286


(                           train  label
 11272     정해 시간을 가지고 찬찬히 보는 것도 한      1
 11273  시간을 가지고 찬찬히 보는 것도 한 방법이다.      0
 11274    가지고 찬찬히 보는 것도 한 방법이다. #      0
 11275       찬찬히 보는 것도 한 방법이다. ##      0
 11276          보는 것도 한 방법이다. ###      1,
                                     train  label
 11277              ### 디자인에 관심이 많은 관람객이라면      1
 11278        ## 디자인에 관심이 많은 관람객이라면 이탈리아관을      1
 11279     # 디자인에 관심이 많은 관람객이라면 이탈리아관을 반드시      0
 11280  디자인에 관심이 많은 관람객이라면 이탈리아관을 반드시 포함해서      1
 11281   관심이 많은 관람객이라면 이탈리아관을 반드시 포함해서 스위스      1)

In [9]:
sentence = train_df.train
sentence[:10]

0              ### 이건 테스트 문장입니다 #
1              ## 이건 테스트 문장입니다 ##
2              # 이건 테스트 문장입니다 ###
3              ### 엠마누엘 웅가로 / 의상서
4            ## 엠마누엘 웅가로 / 의상서 실내
5      # 엠마누엘 웅가로 / 의상서 실내 장식품으로…
6    엠마누엘 웅가로 / 의상서 실내 장식품으로… 디자인
7      웅가로 / 의상서 실내 장식품으로… 디자인 세계
8       / 의상서 실내 장식품으로… 디자인 세계 넓혀
9       의상서 실내 장식품으로… 디자인 세계 넓혀 #
Name: train, dtype: object

In [10]:
labels = train_df.label.values
labels

array([1, 1, 1, ..., 0, 0, 1], dtype=int64)

In [11]:
# text tokenizer setting
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentence]

sentence[0], tokenized_texts[0]

('### 이건 테스트 문장입니다 #', ['#', '#', '#', '이건', '테스트', '문장', '##입니다', '#'])

In [12]:
# token embeding
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# sentence를 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_SEQ_LEN, dtype="int64", truncating="post", padding="post")

input_ids[0]

array([    7,     7,     7,  5370,  7453,  6265, 12190,     7,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [13]:
# attention masks initializer
attention_masks = []

for seq in input_ids :
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

attention_masks[0]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [14]:
# train, validation 분리 -> train : 8, validation : 1, test : 1
train_text, val_text, train_label, val_label = train_test_split(
    input_ids,
    labels,
    random_state=0,
    test_size=TEST_SIZE
)

# attention_mask train, validation 분리
train_masks, val_masks, _, _ = train_test_split(
    attention_masks,
    input_ids,
    random_state=0,
    test_size=TEST_SIZE
)

# data to pytorch_tensor
train_text = torch.tensor(train_text)
train_label = torch.tensor(train_label)
train_masks = torch.tensor(train_masks)
val_text = torch.tensor(val_text)
val_label = torch.tensor(val_label)
val_masks = torch.tensor(val_masks)

print(train_text[0])
print(train_label[0])
print(train_masks[0])
print(val_text[0])
print(val_label[0])
print(val_masks[0])

tensor([ 4297,  2079, 11826,  2069,  4239,  4216,  2205,  2118,  3818,  2088,
         1513,  4000,  5793,  2079,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [15]:
# pytorch learning
train_data = TensorDataset(train_text, train_masks, train_label)
train_sample = RandomSampler(train_data)
train_dataloader = DataLoader(
    train_data, 
    sampler=train_sample, 
    batch_size=TRAIN_BATCH_SIZE
)

validation_data = TensorDataset(val_text, val_masks, val_label)
validation_sample = RandomSampler(validation_data)
validation_dataloader = DataLoader(
    validation_data, 
    sampler=validation_sample,
    batch_size=TRAIN_BATCH_SIZE
)

In [16]:
# test_set setting
sentence = test_df.train
sentence[:10]

11277                  ### 디자인에 관심이 많은 관람객이라면
11278            ## 디자인에 관심이 많은 관람객이라면 이탈리아관을
11279         # 디자인에 관심이 많은 관람객이라면 이탈리아관을 반드시
11280      디자인에 관심이 많은 관람객이라면 이탈리아관을 반드시 포함해서
11281       관심이 많은 관람객이라면 이탈리아관을 반드시 포함해서 스위스
11282     많은 관람객이라면 이탈리아관을 반드시 포함해서 스위스 프랑스관이
11283    관람객이라면 이탈리아관을 반드시 포함해서 스위스 프랑스관이 좋고,
11284       이탈리아관을 반드시 포함해서 스위스 프랑스관이 좋고, 관광은
11285           반드시 포함해서 스위스 프랑스관이 좋고, 관광은 호주
11286          포함해서 스위스 프랑스관이 좋고, 관광은 호주 뉴질랜드
Name: train, dtype: object

In [17]:
labels = test_df.label.values
labels

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [18]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentence]

# sentence[0]
tokenized_texts[0]

['#', '#', '#', '디자인', '##에', '관심', '##이', '많', '##은', '관람객', '##이', '##라면']

In [19]:
input_texts = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_texts = pad_sequences(input_texts, maxlen=MAX_SEQ_LEN, dtype="int64", truncating="post", padding="post")

input_texts[0]

array([    7,     7,     7,  4482,  2170,  3867,  2052,  1039,  2073,
        8369,  2052, 19669,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [20]:
attention_masks = []

for seq in input_texts :
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

attention_masks[0]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [21]:
test_texts = torch.tensor(input_texts)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_texts[0])
print(test_labels[0])
print(test_masks[0])

tensor([    7,     7,     7,  4482,  2170,  3867,  2052,  1039,  2073,  8369,
         2052, 19669,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [22]:
test_data = TensorDataset(test_texts, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=TEST_BATCH_SIZE)

In [23]:
NUM_LABELS = len(train_df.label.unique())

# 각층마다 dropout, dence_unit, activation_function 설정
config = BertConfig.from_pretrained(
    BERT_MODEL_NAME,                    # 사용할 model name
    num_labels=NUM_LABELS,              # label의 갯수
    hidden_dropout_prob=DROPOUT_RATE    # Drop 비율
    # hidden_act="relu"                   # activation_function
)
model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_NAME, 
    config=config
)
model.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [24]:
# optimizer setting
optimizer = AdamW(
    model.parameters(),
    lr = LEARNING_RATE,             # learning_rate
    eps = 1e-8                      # 0으로 나누는 것을 방지하기 위한 epsilon_value
)

# early_stop
# early_stopping = EarlyStopping(
#     monitor = "val_loss",           # 모니터링할 지표
#     mode = MODE,                    # 최대, 최소
#     patience = PATIENCE,            # 몇 번 수치가 개선되지 않으면 종료
#     verbose = False                # 해당 객체의 로그를 표현 여부
# )

# train_step : batch count * epochs
total_steps = len(train_dataloader) * EPOCHS

# learning_rate 조절
schdeuler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

c:\Users\lklkk\anaconda3\envs\sejong\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
# flat_accuracy function
def flat_accuracy(preds, labels) :
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [26]:
# time function
def format_time(elapsed) :
    elapsed_rounded = int(round((elapsed)))

    return str(datetime.timedelta(seconds=elapsed_rounded))

In [28]:
# 재현을 위해 랜덤시드 고정
random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
torch.manual_seed(SEED_VAL)
torch.cuda.manual_seed_all(SEED_VAL)

# gradant initializer
model.zero_grad()
best_loss = float("inf")
counter = 0

# 모델 학습 및 검증 시작
for epoch_i in range(EPOCHS) :
    '''
        Train
    '''
    print("")
    print("======================== Epoch {:} / {:} =========================".format(epoch_i + 1, EPOCHS))
    print("Training...")

    start = time.time()

    # loss initializer
    total_loss = 0

    # train mode change
    model.train()

    # dataloader from batch get data
    for step, batch in enumerate(train_dataloader) :
        if step % SETP_SIZE == 0 and not step == 0 :
            elapsed = format_time(time.time() - start)
            
            print("\tBatch {:} of {:}.\t\tElapsed: {:}.".format(step, len(train_dataloader), elapsed))

        # GPU in batch
        batch = tuple(t.to(device) for t in batch)

        # batch to data
        b_input_texts, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(
            b_input_texts,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels
        )

        # loss calculator
        loss = outputs[0]
        print(loss)

        # total loss
        total_loss += loss.item()

        # BackWard
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # weigth update
        optimizer.step()

        # schduler update
        schdeuler.step()

        # gradent initalizer
        model.zero_grad()

    # avg loss
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("\tAverage training loss : {0:.2f}".format(avg_train_loss))
    print("\tTraining epcoh took : {:}".format(format_time(time.time() - start)))

    break

    '''
        Validation
    '''
    print("")
    print("Running Validation...")

    start = time.time()

    # model 평가
    model.eval()

    # value initalizer
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader :
        batch = tuple(t.to(device) for t in batch)

        b_input_texts, b_input_mask, b_labels = batch

        with torch.no_grad() :
            outputs = model(
                b_input_texts,
                token_type_ids=None,
                attention_mask=b_input_mask,
            )

        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()

        # output logits vs label accuracy
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("\tAccuracy : {0:.2f}".format(eval_accuracy / nb_eval_steps))
    print("\tValidation took : {:}".format(format_time(time.time() - start)))

    # early stop
    if avg_train_loss < best_loss :
        best_loss = avg_train_loss
        counter = 0
        print(best_loss)
    else :
        counter += 1

        if counter >= PATIENCE :
            print("Early stopping")
            break

print("")
print("Training complete!")


======================== Epoch 1 / 50 =========================
Training...
tensor(0.5187, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6632, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6663, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4248, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8139, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.5727, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6172, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6761, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.7701, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.5874, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8421, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4411, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.5272, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.7845, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4843, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3874, device='cuda:0', grad_f

In [ ]:
# test_set evalation
start = time.time()

# eval mode
model.eval()

# variable initalizer
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# test dataloader batch_size loop
for step, batch in enumerate(test_dataloader) :
    if step % SETP_SIZE == 0 and not step == 0 :
        elapsed = format_time(time.time() - start)
        
        print("\tBatch {:} of {:}.\tElapsed : {:}.".format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)

    b_input_texts, b_input_mask, b_labels = batch

    with torch.no_grad() :
        outputs = model(
            b_input_texts,
            token_type_ids=None,
            attention_mask=b_input_mask
        )

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to("cpu").numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy : {0:.2f}".format(eval_accuracy / nb_eval_steps))
print("Test took : {:}".format(format_time(time.time() - start)))

In [ ]:
# 학습 모델 저장
torch.save(model.state_dict(), OUT_MODEL_PATH)

In [ ]:
# 저장한 모델 불러오는 클래스
class KeywordExtractorModel(nn.Module) :
    def __init__(self) :
        super(KeywordExtractorModel, self).__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x) :
        x = self.layer(x)

        return x

In [ ]:
# input data convert
def convert_input_data(sentence) :
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentence]

    # token embeding
    input_texts = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    input_texts = pad_sequences(
        input_texts,
        maxlen=MAX_SEQ_LEN, 
        dtype="int64", 
        truncating="post", 
        padding="post"
    )
    attention_masks = []

    for seq in input_texts :
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)
    
    inputs = torch.tensor(input_texts)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# sentence teset
def test_sentence(sentence) :
    model.eval()

    inputs, masks = convert_input_data(sentence)

    b_input_texts = inputs.to(device)
    b_input_masks = masks.to(device)

    with torch.no_grad() :
        outputs = model(
            b_input_texts,
            token_type_ids=None,
            attention_mask=b_input_masks
        )
    
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
# test sentence 생성
logits = test_sentence(["연기는 별로지만 재미 하나는 끝내줌!"])

print(logits)
print(np.argmax(logits))

In [ ]:
# 저장한 model 불러오기 + 위의 문장 테스트 함수 이용해서 적용되는지 확인 가능.
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = KeywordExtractorModel().to(device)

# model_state_dict = torch.load(out_dir, map_location=device)
# model.load_state_dict(model_state_dict)

# print(model)